Getting started with Spacy<br>
Import data.<br>
We repeat the preprocessing from the previous homework.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os

In [4]:
os.chdir( "/content/gdrive/MyDrive/flair" ) 

In [5]:
path_to_data = os.getcwd() + '/Entity Recognition in Resumes.json'

In [6]:
myfile = open( path_to_data, "r", encoding = "utf-8" )

imported_data = []

for datum in myfile:

    imported_data.append(datum)

myfile.close()

# print first line
print(imported_data[0])
# print how many resumees were read in
print(len(imported_data))

{"content": "Afreen Jamadar\nActive member of IIIT Committee in Third year\n\nSangli, Maharashtra - Email me on Indeed: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6\n\nI wish to use my knowledge, skills and conceptual understanding to create excellent team\nenvironments and work consistently achieving organization objectives believes in taking initiative\nand work to excellence in my work.\n\nWORK EXPERIENCE\n\nActive member of IIIT Committee in Third year\n\nCisco Networking -  Kanpur, Uttar Pradesh\n\norganized by Techkriti IIT Kanpur and Azure Skynet.\nPERSONALLITY TRAITS:\n• Quick learning ability\n• hard working\n\nEDUCATION\n\nPG-DAC\n\nCDAC ACTS\n\n2017\n\nBachelor of Engg in Information Technology\n\nShivaji University Kolhapur -  Kolhapur, Maharashtra\n\n2016\n\nSKILLS\n\nDatabase (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT\nACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)\n\nADDITIONAL INFORMATION\n\nTECHNICAL SKILLS:\n\n• 

In [7]:
import json

In [8]:
mapped_data = [ json.loads( datum ) for datum in imported_data  ]

In [9]:
## data conversion method
def convert_data(data):
    """
    Creates NER training data in Spacy format from JSON dataset
    Outputs the Spacy training data which can be used for Spacy training.
    """
    text = data['content']
    entities = []
    if data['annotation'] is not None:
        for annotation in data['annotation']:
            # only a single point in text annotation.
            point = annotation['points'][0]
            labels = annotation['label']
            # handle both list of labels or a single label.
            if not isinstance(labels, list):
                labels = [labels]
            for label in labels:
                # dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                entities.append((point['start'], point['end'] + 1, label))
    return (text, {"entities": entities})
   
## using a loop or list comprehension, convert each resume in mapped_data using the convert function above, 
## storing the result
converted_resumes = []
for data in mapped_data:
  converted_resumes.append(convert_data(data))
## print the number of resumes in converted resumes 
print(len(converted_resumes))

701


In [10]:
# filter out the resumees whose entities have no entries.
converted_complete_resumees = []
for converted_resumee in converted_resumes:
  entities = converted_resumee[1]["entities"]
  if entities:
    converted_complete_resumees.append(converted_resumee)

Up until now, you could reuse the code from the previous notebook, now, something new comes.

In [11]:
import spacy
nlp = spacy.load('en')
print(nlp)

__nlp__ is Spacy's English language model. For this model, a pretrained NER-model exists.

In [12]:
ner = nlp.get_pipe('ner')
labels = ner.labels
print(labels)

('CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART')


In [13]:
print(spacy.explain("GPE"))
print(spacy.explain("FAC"))
print(spacy.explain("NORP"))

Countries, cities, states
Buildings, airports, highways, bridges, etc.
Nationalities or religious or political groups


__Question 1__: Explain the labels __GPE__, __FAC__, __NORP__.<br>
Which of these labels from __ner__ do you think will Spacy recognize in the resumees?<br>
__Answer 1__:<br>

*   __GPE__: Countries, cities, states
*   __FAC__: Buildings, airports, highways, bridges, etc.
*   __NORP__: Nationalities or religious or political groups

My prediction is that __ner__ recognizes __GPE__ because of cities of universities and __NORP__ because some people may include their nationality. Not so much __FAC__ as these constructs are not very likely to appear in a resumee.

__Task 1__: choose a resumee.

In [14]:
# get a single resume text and print it out.
restxt = converted_complete_resumees[0][0]
## print it out, removing extraneous spaces
print("\n".join(restxt.split('\n\n')))

Afreen Jamadar
Active member of IIIT Committee in Third year
Sangli, Maharashtra - Email me on Indeed: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6
I wish to use my knowledge, skills and conceptual understanding to create excellent team
environments and work consistently achieving organization objectives believes in taking initiative
and work to excellence in my work.
WORK EXPERIENCE
Active member of IIIT Committee in Third year
Cisco Networking -  Kanpur, Uttar Pradesh
organized by Techkriti IIT Kanpur and Azure Skynet.
PERSONALLITY TRAITS:
• Quick learning ability
• hard working
EDUCATION
PG-DAC
CDAC ACTS
2017
Bachelor of Engg in Information Technology
Shivaji University Kolhapur -  Kolhapur, Maharashtra
2016
SKILLS
Database (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT
ACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)
ADDITIONAL INFORMATION
TECHNICAL SKILLS:
• Programming Languages: C, C++, Java, .net, php.
• Web Designing: HTML, XML

Next, we let __nlp__ process that single resumee.<br>
__Task 2__: print the results in __doc__. For each result, print the underlying text and the label.

In [15]:
doc = nlp(restxt)
# Print the results in doc. For each result, print the text and the label.
for ent in doc.ents:
    print(ent.text, ent.label_)

Afreen Jamadar PERSON
IIIT Committee ORG
Third year DATE
Sangli ORG
Maharashtra - Email ORG
IIIT Committee ORG
Third year DATE
Cisco Networking -   ORG
Kanpur GPE
Uttar GPE
Techkriti IIT Kanpur ORG
Azure Skynet WORK_OF_ART
2017 DATE
Bachelor of Engg in Information Technology ORG
Shivaji University ORG
Kolhapur PERSON
Kolhapur GPE
Maharashtra ORG
2016 DATE
SKILLS ORG
Less than 1 year DATE
Less than 1 year DATE
Linux PERSON
Less than 1 year DATE
MICROSOFT ORG
ACCESS ORG
Less than 1 year DATE
MICROSOFT ORG
Less than 1 year DATE
C++ CARDINAL
Java PERSON
XML ORG
Operating Systems ORG
2003 DATE
Linux PERSON
MS Access ORG
SQL ORG
2008 DATE
10 CARDINAL
MySql PERSON


__Question 2__: How well did Spacy perform at recognizing the labels for this text?<br>
__Answer 2__: Not very well, e.g. Java is not a person and C++ not a cardinal.<br>
When Spacy predicted the labels for this resumee, a pretrained model was used.<br>
__Task 3__: print for this resumee the original labels and their corresponding text.

In [17]:
# print for that resumee the original labels and their corresponding text.

labeled_ents = []

for start, end, label in converted_complete_resumees[0][1]['entities']:
  labeled_ents.append((label, restxt[start:end]))

for ent in labeled_ents:
  print(ent)

('Email Address', 'indeed.com/r/Afreen-Jamadar/8baf379b705e37c6')
('Links', 'https://www.indeed.com/r/Afreen-Jamadar/8baf379b705e37c6?isid=rex-download&ikw=download-top&co=IN')
('Skills', 'Database (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT\nACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)\n\nADDITIONAL INFORMATION\n\nTECHNICAL SKILLS:\n\n• Programming Languages: C, C++, Java, .net, php.\n• Web Designing: HTML, XML\n• Operating Systems: Windows […] Windows Server 2003, Linux.\n• Database: MS Access, MS SQL Server 2008, Oracle 10g, MySql.')
('Graduation Year', '2016')
('College Name', 'Shivaji University Kolhapur ')
('Degree', 'Bachelor of Engg in Information Technology')
('Graduation Year', '2017\n')
('College Name', 'CDAC ACTS')
('Degree', 'PG-DAC')
('Companies worked at', 'Cisco Networking')
('Email Address', 'indeed.com/r/Afreen-Jamadar/8baf379b705e37c6')
('Location', 'Sangli')
('Name', 'Afreen Jamadar')


__Question 3__: Compare the performance of the pretrained model __nlp__ and the true labels. Did Spacy perform well? If not, try to explain why.<br>
__Answer 3__: Spacy did not perform very well, because the pretrained model interprets the labels differently as they are meant by the resumee annotations and additionally, some labels which were used are not meaningful for resumees at all.<br>
__Task 4__: Remember last homework? You chose three labels. Select all the resumees, in which all three labels appear.

In [18]:
# fill in your chosen labels in chosen_entity_labels
chosen_entity_labels = [ "Location", "Skills", "Degree" ]

## this method gathers all resumes which have all of the chosen entites above.
def gather_candidates(dataset,entity_labels):
    candidates = list()
    for resume in dataset:
        res_ent_labels = list(zip(*resume[1]["entities"]))[2]
        if set(entity_labels).issubset(res_ent_labels):
            candidates.append(resume)
    return candidates

training_data = gather_candidates( converted_complete_resumees, chosen_entity_labels )
print("Gathered {} training examples".format(len(training_data)))

Gathered 447 training examples


__Task 5__: Next, we want to remove all other entities, since we only want to train NER for the three entities in __chosen_entity_labels__.

In [19]:
## filter all annotation based on filter list
def filter_ents(ents, filter):
    filtered = [ent for ent in ents if ent[2] in filter]
    return filtered

## Use method above to remove all but relevant (chosen) entity annotations and store in X variable. X shall contain all
## the resumees from training_data, but their entity annotations shall be filtered using the function from above.
X = [] 

for text, entities in training_data:
  filtered_ents = filter_ents(entities['entities'], chosen_entity_labels)
  X.append((text, {'entities': filtered_ents}))

__Task 6__: Some resumees cause trouble. We filter these out with the following lines of code.<br>
First, use __add_label__ to add your chosen labels to the __ner__ model.

In [20]:
from spacy.gold import GoldParse 

# add labels 
ner.add_label( chosen_entity_labels[0] )

ner.add_label( chosen_entity_labels[1] )

ner.add_label( chosen_entity_labels[2] )

nlp.begin_training()

good = []

for item in X:
  
  text = nlp.make_doc( item[ 0 ] )

  try:
    
    gold = GoldParse( text, entities = item[ 1 ][ "entities" ] )

  except:

    continue
  
  try:
    
    nlp.update( [ text ], [ gold ], drop = 0.3 )

  except:

    pass

  else:

    good.append( item )

print( "Number of good samples: " + str( len( good ) ) )

print( "" )

print( "" )

print( "Number of bad samples: " + str( len( X ) - len( good ) ) )

Number of good samples: 402


Number of bad samples: 45


For a machine learning model, it is essential to be able to generalize. Only a model, that can generalize well is able to process new data in a meaningful way. Therefore, one usually separates the data set into two sets: the training set and the test set. The training set is used to train the model. The test set is used to evaluate the performance of the model on unseen data.

In [39]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split( [ item[ 0 ] for item in good ], [ item[ 1 ] for item in good ], test_size = 0.3 )

__Task 7__: Complete the following code. Shuffle __new_index__. Create the data sets __x_shuffled__ and __y_shuffled__. Use these to create minibatches, iterate over these minibatches, preprocess the data in a given minibatch using __nlp.make_doc__ and __GoldParse__. Employ __nlp.update__ to update the model using these preprocessed data.

In [114]:
import numpy as np

nlp.begin_training()

new_index = np.arange( len( x_train ) )

x_data = np.array( x_train )

y_data = np.array( y_train )

minibatch_size = 32
epochs = 20

for i in range( epochs ):

  # shuffle new_index
  np.random.shuffle(new_index)

  x_shuffled = x_data[new_index] # create x_shuffled from x_data by using the shuffled new_index

  y_shuffled = y_data[new_index] # create y_shuffled from y_data by using the shuffled new_index

  # divide the data in x_shuffled and y_shuffled into minibatches of identical size
  indices = list(range(minibatch_size, len(x_train), minibatch_size))

  x_minibatches = np.split(x_shuffled, indices)
  y_minibatches = np.split(y_shuffled, indices)

  # iterate over these minibatches
  for x_minibatch, y_minibatch in zip(x_minibatches, y_minibatches):
    # preprocess the data in a minibatch using nlp.make_doc and GoldParse
    doc = nlp.make_doc(str(x_minibatch[0]))
    
    gold = GoldParse(doc, entities=y_minibatch[0]['entities'])

    # use these preprocessed data and nlp.update to train the model
    nlp.update([doc], [gold])

__Question 4__: Why did we shuffle the data?<br> 
Why did we employ mini batches?<br>
Research the term __epoch__ in machine learning. How many epochs of training do we employ?<br>
__Answer 4__: We shuffle the data to reduce the probability of getting mini batches which do not represent the overall data distribution. The use of mini batches relies in the property of being a compromise between SGD and batch gradient decent, means that we have a relatively fast convergence while maintaining enough noise to each gradient update. The __epochs__ are the number of rounds the learning algorithm will work through the entire training dataset to reduce the loss. 20 epochs are employed here.<br>
__Task 8__: Next, we choose one resumee and print it.

In [115]:
resume = x_test[0]

print(resume)

Rayees Parwez
Store Manager - Sales & Operations

Mumbai, Maharashtra - Email me on Indeed: indeed.com/r/Rayees-Parwez/a2c576bd71658aca

Looking part time job in Mumbai

Willing to relocate: Anywhere

WORK EXPERIENCE

Store Manager - Sales & Operations

Al Dukan Retailing company(FMCG) (Dabbagh Group(Mecca -

November 2015 to February 2017

Job profile:

As a Store Manager-Sales I am responsible for sales, Handling, Motivating, Planning, Staffing,
Inventory, Shrinkage & store Hygiene.

Job Description:
. To achieve Targets
. To Maintain Inventory and control Shrinkage
. To ensure company policy and procedures are communicated in a better manner
Followed & practiced and maintaining the store with good merchandising.
. To ensure coordination and out come from team and ensuring team achieves tgts.
. To ensure timely review and feedback takes place for all employees.

Tasks Undertaken:
. Training & Mentoring
. Manpower Planning
. Merchandising
. Operations

Area Sales Manager

Merik Health

__Task 9__: we process this resumee using __nlp__. Print for all items in __doc.ents__ the predicted label and the corresponding text. Then print the correct labels and their corresponding text for that resumee with data from __y_test__.

In [116]:
doc = nlp( resume )
# print for all the items in doc.ents the predicted label and the corresponding text
for item in doc.ents:
  print(item.label_, ":", item.text)

# print the correct labels and their corresponding text for that resumee with data from y_test
print()
labeled_ents = []

for start, end, label in y_test[0]['entities']:
  labeled_ents.append((label, resume[start:end]))

for ent in labeled_ents:
  print(ent)

Location : Mumbai
Location : Mumbai
Location : Delhi
Location : Delhi
Location : Kolkata
Location : Kolkata
Degree : Diploma in Deck Cadet in Deck Cadet
Location : Delhi
Location : New Delhi
Degree : B.A
Degree : Diploma in computer application
Skills : Business Development
Skills : Relationship management
Skills : Business Development

('Skills', 'Relationship management')
('Skills', 'Business Development')
('Skills', 'Relationship management')
('Skills', 'Business Development')
('Degree', 'B.A')
('Degree', 'Diploma in Deck Cadet in Deck Cadet')
('Skills', 'Business Development')
('Location', 'Mumbai')
('Location', 'Mumbai')


__Question 5__: What labels did the model predict correctly?<br> 
Where appeared problems?<br> 
How can you explain the problems?<br>
__Answer 5__: The model predicted "Degree" correctly, misslabeled some "Locations" and did not recognize every "Skill". <br>
__Question 6__: We can evaluate the performance of the model using 4 metrics: the __Accuracy__, the __Precision__, the __Recall__ and __F1__.<br>
Inform yourself on these metrics. How are they defined? Explain the concept of __True Positive__, __True Negative__, __False Positive__ and __False Negative__. Use these to define  the __Accuracy__, the __Precision__, the __Recall__ and __F1__, and also give the formula for each of these.<br>
__Answer 7__: <br>
*   __True Positive__: An outcome where the model *correctly* predicts the *positive* class.
*   __True Negative__: An outcome where the model *correctly* predicts the *negative* class.
*   __False Positive__: An outcome where the model *incorrectly* predicts the *positive* class.
*   __False Negative__: An outcome where the model *incorrectly* predicts the *negative* class.
*   __Accuracy__: Measures the proportion of a model's correct predictions. <br>
 *(TP + TN) / (TP + TN + FP + FN)*
*   __Precision__: Measures the proportion positive identifications that were actually correct. <br>
 *TP / (TP + FP)*
*   __Recall__: Measures the proportion of actual positives that were identified correctly.<br>
 *TP / (TP + FN)*
*   __F1__: An overall measure of a model’s accuracy that combines precision and recall. <br>
 *2 * (Precision * Recall) / (Precision + Recall)*


__Task 10__: Complete the following code. Call __make_bilou_df__ with a resume from the test set and store result in __bilou_df__ variable.

In [117]:
from spacy.gold import biluo_tags_from_offsets
import pandas as pd
from IPython.display import display, HTML

## returns a pandas dataframe with tokens, prediction, and true (Gold Standard) annotations of tokens
def make_bilou_df(nlp,resume):
    """
    param nlp - a trained spacy model
    param resume - a resume from our train or test set
    """
    doc = nlp(resume[0])
    bilou_ents_predicted = biluo_tags_from_offsets(doc, [(ent.start_char,ent.end_char,ent.label_)for ent in doc.ents])
    bilou_ents_true = biluo_tags_from_offsets(doc, [(ent[0], ent[1], ent[2]) for ent in resume[1]["entities"]])

    
    doc_tokens = [tok.text for tok in doc]
    bilou_df = pd.DataFrame()
    bilou_df["Tokens"] =doc_tokens
    bilou_df["Tokens"] = bilou_df["Tokens"].str.replace("\\s+","") 
    bilou_df["Predicted"] = bilou_ents_predicted
    bilou_df["True"] = bilou_ents_true
    return bilou_df

## call method above with a resume from test set and store result in bilou_df variable.
bilou_df = make_bilou_df( nlp, (x_test[0], y_test[0]) )
display(bilou_df)  

,Tokens,Predicted,True
0,Rayees,O,O
1,Parwez,O,O
2,,O,O
3,Store,O,O
4,Manager,O,O
...,...,...,...
914,Development,L-Skills,L-Skills
915,,O,O
916,⇒,O,O
917,Relationship,O,B-Skills


Inform yourself on the [BILUO](https://spacy.io/usage/linguistic-features#accessing-ner) scheme.<br>
__Question 7__: Why do you think is it better to tag entities using this scheme (consider names of humans, descriptions of items in a shop)?<br>
__Answer 7__: Get a more intuitive feeling of wrong predictions.<br>
__Task 11__: employ pandas dataframe api to get a subset where predicted and true labels are the same. Compute the accuracy using the formula you researched above.

In [118]:
## bilou_df is a pandas dataframe. Use pandas dataframe api to get a subset where predicted and true are the same. 
same_df = bilou_df[bilou_df['Predicted'] == bilou_df['True']]
## compute the accuracy
accuracy = len(same_df) / len(bilou_df)

print("Accuracy on one resume: ",accuracy)

Accuracy on one resume:  0.9836779107725789


The __accuracy__ is not 100%. Therefore, we want to have a look at those tokens, where the predicted and the true value differ.<br>
__Task 12__: create a dataframe diff_df where the predicted values and the true values differ

In [119]:
# create a dataframe diff_df where the predicted values and the true values differ
diff_df = bilou_df[bilou_df['Predicted'] != bilou_df['True']]
display(diff_df)

,Tokens,Predicted,True
196,Delhi,U-Location,O
198,Delhi,U-Location,O
393,Kolkata,U-Location,O
684,Business,O,B-Skills
685,Development,O,L-Skills
694,Kolkata,U-Location,O
815,Delhi,U-Location,O
818,New,B-Location,O
819,Delhi,L-Location,O
844,Diploma,B-Degree,O


Since we only considered one resumee, we now make this comparison for the whole test set.<br>
__Task 13__: Complete the following code for the computation of the overall accuracy.

In [120]:
doc_accuracy = []

for i in range( len( x_test ) ):

  resume = (x_test[i], y_test[i])

  bilou_df = make_bilou_df(nlp, resume)

  same_df = bilou_df[bilou_df['Predicted'] == bilou_df['True']]

  doc_accuracy.append( len(same_df) / len(bilou_df) )

total_acc = np.mean( doc_accuracy )
print("Accuracy: ",total_acc)

Accuracy:  0.9040195953284105


So we got an __accuracy__ of about 90% on average. This is quite good considering, that we only considered about 300 cases for training.<br>
__Task 14__: Next, we want to find out, what the model did, when it went wrong. We only consider 5 resumees.<br>
Complete the following code.

In [121]:
for i in range( 5 ):

  resume = (x_test[i], y_test[i])

  bilou_df = make_bilou_df(nlp, resume)

  difference_df = bilou_df[bilou_df['Predicted'] != bilou_df['True']]

  # print, where the labels from Spacy and the annotation differ. Print the text, the predicted and the true labels.
  print(difference_df, '\n')

           Tokens   Predicted      True
196         Delhi  U-Location         O
198         Delhi  U-Location         O
393       Kolkata  U-Location         O
684      Business           O  B-Skills
685   Development           O  L-Skills
694       Kolkata  U-Location         O
815         Delhi  U-Location         O
818           New  B-Location         O
819         Delhi  L-Location         O
844       Diploma    B-Degree         O
845            in    I-Degree         O
846      computer    I-Degree         O
847   application    L-Degree         O
917  Relationship           O  B-Skills
918    management           O  L-Skills 

        Tokens Predicted      True
162       Hard         O  B-Skills
163    working         O  L-Skills
167  Excellent         O  B-Skills
168   learning         O  I-Skills
169    ability         O  I-Skills
..         ...       ...       ...
313   Computer  I-Skills         O
314     Skills  I-Skills         O
315             I-Skills         O
316     

__Question 8__: What was predicted, when the prediction differed from the true label?<br>
What do you think is necessary for computing the accuracy on token level?<br> 
What is the advantage of computing the accuracy on token level?<br>
__Answer 8__: The tokens were predicted as false negatives and false positive so we need to compute the precision and recall to get a better picture of the distribution.<br>
__Task 15__: Complete the following code.

In [125]:
## cycle through chosen_entity_labels and calculate metrics for each entity using test data
data = []
for label in chosen_entity_labels:

  ## variables to store results for all resumes for one entity type
  true_positives = 0
  false_positives = 0
  false_negatives = 0
  for i in range( len( x_test ) ):
    ## use make_bilou_df on each resume in our test set, and calculate for each entity true and false positives,
    ## and false negatives. 

    resume = (x_test[i], y_test[i])
    
    tres_df = make_bilou_df(nlp, resume)

    ## calculate true false positives and false negatives for each resume

    tres_label_df = tres_df[tres_df['Predicted'].str.contains(label) | tres_df['True'].str.contains(label)]

    tp = len(tres_label_df[tres_label_df['Predicted'] == tres_label_df['True']])
    
    fp = len(tres_label_df[(tres_label_df['Predicted'].str.contains(label)) 
      & (tres_label_df['True'].str.contains(label) == False)])
    
    fn = len(tres_label_df[(tres_label_df['Predicted'].str.contains(label) == False) 
      & (tres_label_df['True'].str.contains(label))])

    ## aggregate result for each resume to totals
    true_positives = true_positives + tp
    false_positives = false_positives + fp
    false_negatives = false_negatives + fn
    
  print("For label '{}' tp: {} fp: {} fn: {}".format(label,true_positives,false_positives,false_negatives))
  
  ## Use the formulas you learned to calculate metrics and print them out
  ## also: prevent division by zero without raising errors. Explain your choice

  if (true_positives == 0):
    precision = 0
    recall = 0
    f1 = 0
  else:
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1 = 2 * (precision * recall) / (precision + recall)

  row = [label,precision,recall,f1]
  data.append(row)

## make pandas dataframe with metrics data. Use the chosen entity labels as an index, and the metric names as columns. 
metric_df = pd.DataFrame( data, columns = [ "Label", "Precision", "Recall", "F1" ] )
display(metric_df)

For label 'Location' tp: 265 fp: 184 fn: 63
For label 'Skills' tp: 1186 fp: 1916 fn: 1523
For label 'Degree' tp: 432 fp: 352 fn: 59


,Label,Precision,Recall,F1
0,Location,0.590200,0.807927,0.682111
1,Skills,0.382334,0.437800,0.408191
2,Degree,0.551020,0.879837,0.677647


__Question 9__: Explain from these statistics how well __nlp__ performs.<br>
__Answer 9__: The overall performance is not very good, e.g. for "Skills" only around 40% (F1) were predicted correctly. <br>
__Task 16__: Compute for each metric (Precision, Recall, F1) the mean values.

In [126]:
for label in [ "Precision", "Recall", "F1" ]:
    
    # Compute mean and print
    print(label, ": ", np.mean(metric_df[label]))

Precision :  0.5078516105587408
Recall :  0.7085212742166961
F1 :  0.5893163673819023


__Question 10__: What do you learn, when you compare the performance of the model on the token level with the performance of the model on the global level from above?<br>
__Answer 10__: The performance for each token varies regarding the precision as well as recall. <br>
Next, we prepare data for flair.

In [128]:
train = [ [ x_train[ i ], y_train[ i ] ] for i in range( len( x_train ) ) ]

test = [ [ x_test[ i ], y_test[ i ] ] for i in range( len( x_test ) ) ]

__Task 17__: Complete the following code.

In [109]:
# prepare data
training_data_as_bilou = [make_bilou_df(nlp,res) for res in train]

test_data_as_bilou = [make_bilou_df(nlp,res) for res in test]


# set up paths
path_to_training_file = os.getcwd() + "/training_data.csv"

path_to_test_file = os.getcwd() + "/test_data.csv"



# make sure, that if the corresponding files exist, they are emptied
if os.path.isfile( path_to_training_file ):

  open( path_to_training_file, "w" ).close()

if os.path.isfile( path_to_test_file ):

  open( path_to_test_file, "w" ).close()


# open empty files
training_file = open( path_to_training_file, "a", encoding = "utf-8" )
    
test_file = open( path_to_test_file, "a", encoding = "utf-8" )


for item in training_data_as_bilou:
  # remove all tokens like "", " ", "\n" by ignoring them
  # for all other tokens do the following:
  # create a string s: s = token + " " + label + "\n"
  # if the label is "-", then write s = token + " O\n"
  #
  # write this newly created string to file
  # if this newly created string contains ".", then also write a 
  # newline to file that only contains "\n"
  #
  # Using this scheme, each line in the resulting files corresponds either to an empty line or a token.
  # Flair assembles a block of nonempty lines into a sentence. Therefore, the empty line
  # is a signal for Flair that the current sentence is finished. Therefore, we extracted
  # the whitespaces above.
  for token, label in zip(item['Tokens'], item['True']):

    if token in ["", " ", "\n"]:
      continue

    s = ""
    if label == "-":
      s = token + " O\n"
    else:
      s = token + " " + label + "\n"

    training_file.write(s)
    
    if "." in s:
      training_file.write("\n")

for item in test_data_as_bilou:

  for token, label in zip(item['Tokens'], item['True']):

    if token in ["", " ", "\n"]:
      continue

    s = ""
    if label == "-":
      s = token + " O\n"
    else:
      s = token + " " + label + "\n"

    test_file.write(s)
    
    if "." in s:
      test_file.write("\n")


training_file.close()

test_file.close()

Start Flair

In [110]:
pip install flair

     |████████████████████████████████| 286kB 5.0MB/s 
     |████████████████████████████████| 2.3MB 19.4MB/s 
     |████████████████████████████████| 798kB 34.2MB/s 
     |████████████████████████████████| 1.2MB 33.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 71kB 6.9MB/s 
     |████████████████████████████████| 19.7MB 1.3MB/s 
     |████████████████████████████████| 776.8MB 24kB/s 
     |████████████████████████████████| 983kB 12.2MB/s 
     |████████████████████████████████| 901kB 30.2MB/s 
     |████████████████████████████████| 3.3MB 28.4MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=1e3d8cbb970fc7d6cbeb4ff161a2aa0a48814dd23c0dac7caf196642ad9a5587
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Created wheel for sqlitedict: file

In [111]:
from flair.data import Corpus
from flair.data_fetcher import NLPTaskDataFetcher


# your training file name
data_folder = os.getcwd() 

train_file = "training_data.csv"

# your test file name
test_file = "test_data.csv"

# when we wrote the data files, each row was either empty to signal the end
# of a sentence to Flair, or the line contained a token, a white space and a label.
# In the next line, we assign, that the token is the "text", and that the label is 
# "ner" label
columns =  {0: 'text', 1: 'ner'}

## Now load our csv into flair corpus
corpus = NLPTaskDataFetcher.load_column_corpus(data_folder,column_format=columns,
                                               train_file=train_file,
                                               test_file=test_file)
print(corpus)

2021-05-20 07:50:30,229 Reading data from /content/gdrive/My Drive/flair
2021-05-20 07:50:30,231 Train: /content/gdrive/My Drive/flair/training_data.csv
2021-05-20 07:50:30,239 Dev: None
2021-05-20 07:50:30,242 Test: /content/gdrive/My Drive/flair/test_data.csv


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated function (or staticmethod) load_column_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.


Corpus: 7975 train + 886 dev + 3032 test sentences
